<a href="https://colab.research.google.com/github/unBAis/AI-Powered-Legal-Document-Assistant-RAG-Based-/blob/main/Rag_Project_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install -q transformers accelerate torch sentencepiece torch

In [1]:
!pip install -q --upgrade huggingface_hub bitsandbytes accelerate transformers torch faiss-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [7]:
!pip install pymupdf requests openpyxl beautifulsoup4 pandas selenium webdriver-manager sentencepiece faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.5 MB/s eta 0:00:00


In [26]:
from huggingface_hub import login
login()
# from huggingface_hub import whoami
# print(whoami())

In [9]:
import os
import subprocess
import sys
import json
import re

import requests
from bs4 import BeautifulSoup
import fitz
import json


import pandas as pd
import numpy as np
import sys

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
from transformers import BitsAndBytesConfig
import torch

# Install dependencies
def install_requirements():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])

if __name__ == "__main__":
    install_requirements()

In [10]:

def extract_text_from_pdf(url):
    response = requests.get(url)
    pdf_path = "downloaded.pdf"

    # Save PDF locally
    with open(pdf_path, "wb") as f:
        f.write(response.content)

    # Extract text
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])

    return text


def preprocess_text(text):
    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [11]:
url = "https://www.sci.gov.in/judgements-case-no/"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Extract judgment links
for link in soup.find_all("a", href=True):
    if "pdf" in link["href"]:
        print("Found judgment PDF:", link["href"])

# link["href"][1]

Found judgment PDF: https://judgments.ecourts.gov.in/pdfsearch/?p=pdf_search/index&escr_flag=Y
Found judgment PDF: http://cdnbbsr.s3waas.gov.in/s3ec0490f1f4972d133619a60c30f3559e/documents/misc/practice.pdf_0.pdf


In [12]:

# Download and extract text
pdf_url = "http://cdnbbsr.s3waas.gov.in/s3ec0490f1f4972d133619a60c30f3559e/documents/misc/practice.pdf_0.pdf"
pdf_text = extract_text_from_pdf(pdf_url)

# Convert to JSON
pdf_data = {"url": pdf_url, "content": pdf_text}
json_output = json.dumps(pdf_data, indent=4)

# Save to file
with open("legal_documents.json", "w") as json_file:
    json_file.write(json_output)

print("✅ PDF data saved to `legal_documents.json`")


✅ PDF data saved to `legal_documents.json`


In [13]:
# Load the extracted JSON
with open("legal_documents.json", "r") as f:
    data = json.load(f)

# Apply preprocessing
data["content"] = preprocess_text(data["content"])

# # Save the cleaned version
with open("cleaned_legal_documents.json", "w") as f:
    json.dump(data, f, indent=4)

print("Preprocessed text saved.")


Preprocessed text saved.


In [14]:
# Load preprocessed text
with open("cleaned_legal_documents.json", "r") as f:
    data = json.load(f)

# Load an embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")


# Split long text into chunks (e.g., every 512 tokens)
chunk_size = 512  # Adjust based on model limits

long_text = data["content"]
chunks = [long_text[i:i + chunk_size] for i in range(0, len(long_text), chunk_size)]

# Embed each chunk separately
embeddings = [model.encode(chunk) for chunk in chunks]

print(f"Generated {len(embeddings)} embeddings for {len(long_text)} characters.")

# # Save embeddings
# with open("embedded_legal_documents.json", "w") as f:
#     json.dump(data, f, indent=4)

# print("Embeddings saved.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generated 837 embeddings for 428422 characters.


In [ ]:
type(embeddings)

In [15]:
import numpy as np
val = [0.2364,0.309,0.3636]
sum = 0
exp_list = []
for i in val:
    # print(np.exp(i))
    exp_list.append(np.exp(i))
    sum += np.exp(i)
for i in exp_list:
    print(round(i/sum,4))
# [0.3114, 0.3349,	0.3537]

0.3114
0.3349
0.3537


In [16]:

# Convert embeddings list to numpy array
embedding_array = np.array(embeddings).astype("float32")

# Create FAISS index
index = faiss.IndexFlatL2(embedding_array.shape[1])
index.add(embedding_array)

# Save the index
faiss.write_index(index, "vector_index.faiss")

# each chunk (512 tokens) gets an aggregated vector of 384 dimensions
pd.DataFrame(embedding_array)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.024588,0.052831,-0.014663,-0.099885,-0.099500,0.013548,0.063110,0.082049,-0.019919,0.008487,...,0.019549,0.089404,0.028906,-0.001019,0.003330,-0.015777,0.076196,0.014322,-0.014824,0.024162
1,-0.009339,0.035696,0.049667,-0.107434,-0.050368,-0.004023,-0.019305,0.047309,-0.050787,0.052848,...,0.047694,0.073821,0.014311,0.064436,-0.039369,0.016471,0.127457,0.020829,-0.011706,0.023915
2,0.031597,0.055003,0.046474,-0.100043,0.006485,0.023661,0.044096,-0.038630,0.020649,0.000687,...,0.047901,0.037684,0.047201,-0.030602,-0.031263,0.025404,0.175338,0.057485,0.019360,0.037009
3,-0.021659,0.046918,0.092786,-0.069770,-0.064544,0.074686,-0.004564,-0.010819,-0.073910,0.069663,...,0.041401,0.115750,0.037047,-0.010741,-0.013338,-0.040246,0.082177,-0.021479,-0.077693,-0.045797
4,0.008933,0.115678,0.020985,-0.022641,-0.011192,0.075727,0.040123,0.019425,0.004098,-0.005822,...,0.017384,0.022738,0.041650,-0.048823,-0.012940,-0.034575,0.117087,-0.014593,-0.010254,0.052849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,-0.024460,-0.036082,-0.010086,-0.081071,-0.085857,0.049538,-0.001693,0.002232,0.000384,-0.027242,...,-0.021156,0.068199,0.059592,0.013561,-0.002962,-0.006980,0.072677,-0.017416,0.057388,0.027378
833,-0.065814,0.002947,-0.027804,-0.096675,-0.043336,0.049608,-0.003992,-0.045176,0.026496,-0.075825,...,-0.068381,0.062035,-0.006136,0.043889,0.068750,-0.010470,0.055964,-0.017161,0.062175,0.072650
834,-0.054094,-0.000679,0.002470,-0.056088,-0.043441,0.048455,0.019706,0.060164,0.038288,0.038640,...,-0.050505,0.078354,0.041121,-0.009197,-0.057559,-0.052339,0.069725,-0.029564,0.036384,0.071469
835,0.009620,0.101139,0.036150,-0.057544,-0.015316,-0.002235,-0.005001,-0.056868,-0.015683,-0.019974,...,-0.034907,0.084695,0.071747,-0.009981,-0.000587,0.027678,0.082463,0.008928,0.029630,0.072154


In [17]:
# Search for a Similar Chunk-- rdundancy
query_embedding = model.encode("What is the Supreme Court's verdict?")
query_embedding = np.array([query_embedding]).astype("float32")

# Search for top 3 similar chunks
D, I = index.search(query_embedding, k=3)
print("Top similar chunks:", I)


Top similar chunks: [[ 38 736  37]]


In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(embedding_array)

# Assign cluster labels to chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i} - Cluster {labels[i]}: {chunk[:100]}")


In [19]:
# what all clusters are present
set(labels)

{np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4)}

In [32]:
# optimal bits configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")



config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:

input_text = "Explain Machine Learning to me in a nutshell."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

output = model.generate(input_ids, max_length=100)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

In [34]:

# Format prompt
retrieved_text = " ".join([chunks[i] for i in I[0]])
prompt = f"Use the following context to answer: {retrieved_text}\n\nQ: What is the Supreme Court's decision?"

# Tokenize input
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")  # Move to GPU

# Generate response
output_ids = model.generate(input_ids, max_length=500)
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following context to answer: nvicted the accused person and sentenced him to death; or if the High Court certifies under Article 134A that the case is a fit one for appeal to the Supreme Court, subject to the proviso thereunder. 9 Article 136 Notwithstanding anything contained in Chapter IV of the Constitution, the Supreme Court may in its discretion, grant special leave to appeal from any judgment, decree, determination, sentence or order in any cause or matter passed or made by any court or tribunal, except in relation to any court o  of the Supreme Court a petition of appeal (copy enclosed) from the judgment and decree/order of the High Court of Judicature at …………and the said petition has been registered in Supreme Court as Civil Appeal No. ………of ……..20…………. Notice is hereby given to you that if you wish to contest the appeal you may appear within thirty days of the receipt of this notice before this Court either personally or by an advocate-on-record of the Court appointed 

In [36]:
# output tuning
output_ids = model.generate(input_ids, max_length=500, temperature=0.7, top_p=0.9)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
